# Creation of an open source model for DREAM descriptor prediction

### Preliminaries and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from missingpy import KNNImputer
import mordred
import opc_python.utils.loading as dream_loading
import numpy as np
import pandas as pd
import pickle
from pyrfume.odorants import from_cids, all_smiles
from pyrfume.features import smiles_to_mordred, smiles_to_morgan, smiles_to_morgan_sim
from rickpy import ProgressBar
import rdkit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.multioutput import MultiOutputRegressor

### Load the perceptual data from Keller and Vosshall, 2016

In [3]:
kv2016_perceptual_data = dream_loading.load_raw_bmc_data()
kv2016_perceptual_data = dream_loading.format_bmc_data(kv2016_perceptual_data,
                                                       only_dream_subjects=False, # Whether to only keep DREAM subjects
                                                       only_dream_descriptors=True, # Whether to only keep DREAM descriptors
                                                       only_dream_molecules=True) # Whether to only keep DREAM molecules)

In [4]:
# Get the list of PubChem IDs from this data
kv_cids = list(kv2016_perceptual_data.index.get_level_values('CID').unique())

In [7]:
   in kv_cids

False

In [6]:
# Get information from PubChem about these molecules
info = from_cids(kv_cids)
# Make a Pandas series relating PubChem IDs to SMILES strings
smiles = pd.Series(index=kv_cids, data=[x['IsomericSMILES'] for x in info])
smiles.head()

[-----------------------100%---------------------] 476 out of 476 complete (Retrieved 400 through 475) 

126     C1=CC(=CC=C1C=O)O
176               CC(=O)O
177                  CC=O
180               CC(=O)C
196    C(CCC(=O)O)CC(=O)O
dtype: object

### Compute physicochemical features for the DREAM data (and some other molecules)

In [76]:
# Get a list of all SMILES strings from the Pyrfume library
ref_smiles = list(set(all_smiles()))

In [77]:
mordred_features_ref = smiles_to_mordred(ref_smiles)

[-----------------------100%---------------------] 9762 out of 9762 complete (Finished embedding all molecules)
Computing Mordred features...


  0%|          | 7/9729 [00:08<5:24:27,  2.00s/it] 

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  2%|▏         | 157/9729 [00:15<06:47, 23.49it/s] 

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


...

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 97%|█████████▋| 9468/9729 [09:11<00:11, 21.81it/s]  

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 9729/9729 [09:25<00:00, 17.20it/s]


There are 9729 molecules and 1826 features


In [18]:
# A KNN imputer instance
imputer_knn = KNNImputer(n_neighbors=10, col_max_missing=1)
X = mordred_features_ref.astype(float)
imputer_knn.fit(X)

KNNImputer(col_max_missing=1, copy=True, metric='masked_euclidean',
           missing_values='NaN', n_neighbors=10, row_max_missing=0.5,
           weights='uniform')

In [20]:
# Compute Mordred features from these SMILES strings
mordred_features = smiles_to_mordred(smiles.values)

[-----------------------100%---------------------] 476 out of 476 complete (Finished embedding all molecules)   
Computing Mordred features...


100%|██████████| 476/476 [00:17<00:00, 27.63it/s]


There are 476 molecules and 1826 features


In [42]:
# The computed Mordred features as floats (so errors become NaNs)
X = mordred_features.astype(float)
# Whether a column (one feature, many molecules) has at least 50% non-NaN values
is_good_col = X.isnull().mean() < 0.5
# The list of such "good" columns (i.e. well-behaved features)
good_cols = is_good_col[is_good_col].index
# Impute the missing (NaN) values
X[:] = imputer_knn.fit_transform(X)
# Restrict Mordred features to those from the good columns, even after imputation
X = X[good_cols] 
# Put back into a dataframe
mordred_features_knn = pd.DataFrame(index=mordred_features.index, columns=good_cols, data=X)

/home/rgerkin/dev/missingpy/missingpy/knnimpute.py:143: UserWarning: Insufficient number of neighbors! Filling in column mean.
  warnings.warn("Insufficient number of neighbors! "


In [92]:
# Compute Morgan fingerprint similarities from these SMILES strings
morgan_sim_features = smiles_to_morgan_sim(smiles.values, ref_smiles)

9762 similarity features for 476 molecules


In [93]:
len(ref_smiles), len(set(ref_smiles))

(9762, 9762)

In [94]:
len(list(morgan_sim_features)), len(set(morgan_sim_features))

(9762, 9762)

In [95]:
# Combine Mordred (after imputation) and Morgan features into one dataframe
all_features = mordred_features_knn.join(morgan_sim_features, lsuffix="mordred_", rsuffix="morgan_")
assert len(all_features.index) == len(all_features.index.unique())
assert list(all_features.index) == list(smiles.values)
all_features.index = smiles.index
all_features.index.name = 'PubChem CID'
all_features.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,C/C=C/CCCO,C[C@@H]1CC[C@H]([C@H](C1)NC(=O)C2CC2)C(C)C,COC1=C(C=C(C=C1Br)Br)Br,CC(=O)OC1CC2CC1C3C2CC=C3,CCOC(=O)C(C)C(=O)OCC,CCCCCCC(C)(C)S,CCC=C(C)C(=O)OCC,C=CCOC(=O)COC1=CC=CC=C1,CC(C)(C)NCC(C1=CC(=CC(=C1)O)O)O.CC(C)(C)NCC(C1=CC(=CC(=C1)O)O)O.OS(=O)(=O)O,C1CC(OC1)CCO
PubChem CID,,,,,,,,,,,,,,,,,,,,,
126,6.473351,6.127583,0.0,0.0,11.189957,2.193993,4.387987,11.189957,1.243329,3.089765,...,0.078431,0.023529,0.125000,0.070588,0.029412,0.000000,0.066667,0.209302,0.136054,0.035714
176,2.449490,2.449490,1.0,0.0,3.464102,1.732051,3.464102,3.464102,0.866025,2.178059,...,0.133333,0.125000,0.046512,0.156250,0.170213,0.051282,0.256410,0.092308,0.047619,0.057143
177,1.414214,1.414214,0.0,0.0,2.828427,1.414214,2.828427,2.828427,0.942809,1.849457,...,0.214286,0.064516,0.048780,0.064516,0.088889,0.054054,0.162162,0.063492,0.032258,0.000000
180,2.449490,2.449490,0.0,0.0,3.464102,1.732051,3.464102,3.464102,0.866025,2.178059,...,0.066667,0.156250,0.046512,0.156250,0.212766,0.102564,0.307692,0.092308,0.047619,0.000000
196,6.801520,6.726049,2.0,0.0,10.987918,2.000000,4.000000,10.987918,1.098792,3.123214,...,0.188679,0.068966,0.000000,0.068966,0.228571,0.193548,0.193548,0.113636,0.080537,0.103448


In [96]:
len(list(all_features)), len(set(all_features))

(11425, 11425)

### Organize perceptual data

In [97]:
# Compute the descriptor mean across subjects
data_mean = kv2016_perceptual_data.mean(axis=1)
# Compute the subject-averaged descriptor mean across replicates
data_mean = data_mean.unstack('Descriptor').reset_index().groupby(['CID', 'Dilution']).mean().iloc[:, 1:]
# Fix the index for joining
data_mean.index = data_mean.index.rename(['PubChem CID', 'Dilution'])
# Show the dataframe
data_mean.head()

Descriptor                Acid   Ammonia    Bakery     Burnt   Chemical  \
PubChem CID Dilution                                                      
126         -3.0      5.657143  3.500000  0.400000  1.235294  12.411765   
            -1.0      2.673469  1.480000  0.755102  1.551020  12.600000   
176         -7.0      3.166667  5.000000  0.714286  1.384615  16.500000   
            -5.0      2.090909  4.476190  3.142857  4.272727   5.954545   
177         -5.0      2.541667  0.416667  6.000000  1.708333   8.640000   

Descriptor                Cold   Decayed      Fish    Flower     Fruit  ...  \
PubChem CID Dilution                                                    ...   
126         -3.0      6.142857  3.764706  0.000000  5.676471  2.257143  ...   
            -1.0      1.340000  4.580000  0.510204  7.591837  3.760000  ...   
176         -7.0      0.083333  4.846154  5.846154  6.214286  0.923077  ...   
            -5.0      6.409091  3.761905  0.000000  7.714286  1.523810  ...   
177         -5.0      4.400000  4.083333  2.173913  6.083333  4.695652  ...   

Descriptor               Grass  Intensity     Musky  Pleasantness       Sour  \
PubChem CID Dilution                                                           
126         -3.0      1.794118  37.567568  5.117647     51.837838   2.558824   
            -1.0      1.265306  51.634615  5.120000     49.211538   7.200000   
176         -7.0      5.857143  11.789474  7.466667     46.052632   9.928571   
            -5.0      2.217391  21.241379  7.181818     47.793103  19.545455   
177         -5.0      1.652174  21.812500  8.826087     53.968750   8.840000   

Descriptor              Spices    Sweaty      Sweet      Warm      Wood  
PubChem CID Dilution                                                     
126         -3.0      4.970588  1.117647  15.861111  5.571429  0.235294  
            -1.0      2.489796  1.632653  14.780000  1.100000  1.183673  
176         -7.0      5.333333  8.428571   2.285714  9.533333  6.285714  
            -5.0      9.304348  8.086957   7.173913  5.238095  2.285714  
177         -5.0      0.520000  1.608696  15.923077  2.307692  0.360000  

[5 rows x 21 columns]

### Join the features and the descriptors and split again for prediction

In [98]:
# Create a joined data frame with perceptual descriptors and physicochemical features
df = data_mean.join(all_features, how='inner')
# Add a column for dilution (used in prediction)
df['Dilution'] = df.index.get_level_values('Dilution')
# Make a list of all the columns that will be used in prediction
predictor_columns = [col for col in list(df) if col not in list(data_mean)]
# Make a list of all the columns that must be predicted
data_columns = list(data_mean)
# Create the feature matrix and the target matrix
X = df[predictor_columns]
Y = df[data_columns]

In [101]:
# Each feature name is only used once
assert pd.Series(predictor_columns).value_counts().max()==1

### Verify that this model gets reasonable out-of-sample performance

In [102]:
# A function to compute the correlation between the predicted and observed ratings
# (for a given descriptor columns)
def get_r(Y, Y_pred, col=0):
    pred = Y_pred[:, col]
    obs = Y.iloc[:, col]
    return np.corrcoef(pred, obs)[0, 1]

# A series of scorers, one for each descriptor
scorers = {desc: make_scorer(get_r, col=i) for i, desc in enumerate(Y.columns)}
# The number of splits to use in cross-validation
n_splits = 5
# The number of descriptors in the perceptual data
n_descriptors = Y.shape[1]
# A vanilla Random Forest model with only 10 trees (performance will increase with more trees)
rfr = RandomForestRegressor(n_estimators=10, random_state=0)
# A multioutput regressor used to fit one model per descriptor, in parallel
mor = MultiOutputRegressor(rfr, n_jobs=n_descriptors)
# Check the cross-validation performance of this kind of model
%time cv_scores = cross_validate(mor, X, Y, scoring=scorers, cv=n_splits)

CPU times: user 16.8 s, sys: 6.72 s, total: 23.5 s
Wall time: 5min 32s


In [104]:
# An empty dataframe to hold the cross-validation summary
rs = pd.DataFrame(index=list(Y))
# Compute the mean and standard deviation across cross-validation splits
rs['Mean'] = [cv_scores['test_%s' % desc].mean() for desc in list(Y)]
rs['StDev'] = [cv_scores['test_%s' % desc].std() for desc in list(Y)]
# Show the results
rs

,Mean,StDev
Acid,0.299096,0.075315
Ammonia,0.232537,0.058843
Bakery,0.506021,0.133803
Burnt,0.329004,0.075727
Chemical,0.399673,0.034350
Cold,0.137494,0.075718
Decayed,0.278040,0.060595
Fish,0.398838,0.100292
Flower,0.316520,0.077452
Fruit,0.458896,0.112637


### Fit the final model and save it

In [105]:
# A random forest regressor with more trees
rfr = RandomForestRegressor(n_estimators=250, random_state=0)
# Wrap in a class that will fit one model per descriptor
mor = MultiOutputRegressor(rfr, n_jobs=n_descriptors)
# Fit the model
%time mor.fit(X, Y);

CPU times: user 1.12 s, sys: 4.16 s, total: 5.27 s
Wall time: 35min 39s


MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=250,
                                                     n_jobs=None,
                                                     oob_score=False,
                                                 

In [106]:
len(list(X)), len(set(list(X)))

(11426, 11426)

In [107]:
# Save the fitted model
path = pyrfume.DATA_DIR / 'keller_2017' / 'open-source-dream.pkl'
with open(path, 'wb') as f:
    pickle.dump([mor, list(X), list(Y), imputer_knn], f)

### Demonstration: using the fitted model (can be run independently if the above has been run at some point)

In [1]:
from pyrfume.odorants import from_cids
from pyrfume.predictions import load_dream_model, smiles_to_features, predict
novel_cids = [14896, 228583] # Beta-pinene and 2-Furylacetone
novel_info = from_cids(novel_cids)
novel_smiles = [x['IsomericSMILES'] for x in novel_info]
model_, use_features_, descriptors_, imputer_ = load_dream_model()
features_ = smiles_to_features(novel_smiles, use_features_, imputer_)
predict(model_, features_, descriptors_)

[-----------------------100%---------------------] 2 out of 2 complete (Finished embedding all molecules)
Computing Mordred features...


100%|██████████| 2/2 [00:00<00:00,  3.79it/s]


There are 2 molecules and 1826 features
(2, 1826)


/home/rgerkin/dev/missingpy/missingpy/knnimpute.py:143: UserWarning: Insufficient number of neighbors! Filling in column mean.
  warnings.warn("Insufficient number of neighbors! "


9762 similarity features for 2 molecules


,Acid,Ammonia,Bakery,Burnt,Chemical,Cold,Decayed,Fish,Flower,Fruit,...,Grass,Intensity,Musky,Pleasantness,Sour,Spices,Sweaty,Sweet,Warm,Wood
CC1(C2CCC(=C)C1C2)C,5.87,2.98,1.84,2.36,13.90,5.45,1.04,0.86,4.86,4.97,...,4.66,42.3,5.49,52.55,6.62,5.49,0.68,8.30,6.16,5.56
CC(=O)CC1=CC=CO1,6.65,4.38,2.46,6.39,12.86,3.09,4.16,1.49,3.32,1.96,...,1.91,64.0,8.11,40.83,6.17,4.69,3.31,8.51,5.17,3.02


In [9]:
Out[1].to_dict('records')

[{'Acid': 5.87,
  'Ammonia': 2.98,
  'Bakery': 1.84,
  'Burnt': 2.36,
  'Chemical': 13.9,
  'Cold': 5.45,
  'Decayed': 1.04,
  'Fish': 0.86,
  'Flower': 4.86,
  'Fruit': 4.97,
  'Garlic': 2.69,
  'Grass': 4.66,
  'Intensity': 42.3,
  'Musky': 5.49,
  'Pleasantness': 52.55,
  'Sour': 6.62,
  'Spices': 5.49,
  'Sweaty': 0.68,
  'Sweet': 8.3,
  'Warm': 6.16,
  'Wood': 5.56},
 {'Acid': 6.65,
  'Ammonia': 4.38,
  'Bakery': 2.46,
  'Burnt': 6.39,
  'Chemical': 12.86,
  'Cold': 3.09,
  'Decayed': 4.16,
  'Fish': 1.49,
  'Flower': 3.32,
  'Fruit': 1.96,
  'Garlic': 1.92,
  'Grass': 1.91,
  'Intensity': 64.0,
  'Musky': 8.11,
  'Pleasantness': 40.83,
  'Sour': 6.17,
  'Spices': 4.69,
  'Sweaty': 3.31,
  'Sweet': 8.51,
  'Warm': 5.17,
  'Wood': 3.02}]